### Imports

In [1]:
import os
import os.path as osp
import random
import yaml
import torch
import numpy as np
from tqdm import tqdm

from data import get_dataloaders
from models import load_model
from utils import convert_dict_to_tuple

### Configure

In [2]:
CONFIG_FILE = 'config/resnet18.yml'

### Config and Setup

In [3]:
with open(CONFIG_FILE) as f:
        data = yaml.safe_load(f)
config = convert_dict_to_tuple(data)

seed = config.dataset.seed
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)


outdir = osp.join('./datasets', f'VPRC2023_{config.name}')
print("output directory: {}".format(outdir))
if not os.path.exists(osp.join(outdir, 'train')):
    os.makedirs(osp.join(outdir, 'train'))
if not os.path.exists(osp.join(outdir, 'test')):
    os.makedirs(osp.join(outdir, 'test'))

output directory: ./datasets\VPRC2023_resnet18_baseline


### Data and Model

In [4]:
train_loader, test_loader = get_dataloaders(config)

print("Loading model...")
model = load_model(config)
model.eval()
print("Done!")


Preparing train reader...
Done.
Preparing test reader...
Done.
Loading model...


Using cache found in C:\Users\zzech/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\zzech\anaconda3\envs\colab\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\zzech\anaconda3\envs\colab\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Done!


In [6]:
with torch.no_grad():
    for epoch in range(config.dataset.augmentation.epoch):
        train_iter = tqdm(train_loader, desc='Train Set Feature Extraction', dynamic_ncols=True, position=1)
        for step, batch in enumerate(train_iter):
            x = batch['image']
            y = batch['label']
            names = batch['name']
            out = model(x.cuda().to(memory_format=torch.contiguous_format)).cpu().numpy()

            for i in range(len(x)):
                np.save(os.path.join(outdir, 'train', f"{y[i]:04d}_{epoch}_{names[i]}"), out[i])

    test_iter = tqdm(test_loader, desc='Test Set Feature Extraction', dynamic_ncols=True, position=1)
    for step, batch in enumerate(test_iter):
        x = batch['image']
        y = batch['label']
        names = batch['name']
        out = model(x.cuda().to(memory_format=torch.contiguous_format)).cpu().numpy()

        for i in range(len(x)):
            np.save(os.path.join(outdir, 'test', f"{y[i]:04d}_{names[i]}"), out[i])



Train Set Feature Extraction:   0%|          | 0/34 [00:08<?, ?it/s]


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\zzech\anaconda3\envs\colab\lib\site-packages\torch\utils\data\_utils\worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "C:\Users\zzech\anaconda3\envs\colab\lib\site-packages\torch\utils\data\_utils\fetch.py", line 61, in fetch
    return self.collate_fn(data)
  File "C:\Users\zzech\anaconda3\envs\colab\lib\site-packages\torch\utils\data\_utils\collate.py", line 265, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
  File "C:\Users\zzech\anaconda3\envs\colab\lib\site-packages\torch\utils\data\_utils\collate.py", line 128, in collate
    return elem_type({key: collate([d[key] for d in batch], collate_fn_map=collate_fn_map) for key in elem})
  File "C:\Users\zzech\anaconda3\envs\colab\lib\site-packages\torch\utils\data\_utils\collate.py", line 128, in <dictcomp>
    return elem_type({key: collate([d[key